In [18]:
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3
import category_encoders as ce
from sklearn.feature_selection import chi2, SelectKBest

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.preprocessing import Imputer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import FunctionTransformer

import os
import sys
conn = sqlite3.connect('db/wine_data.sqlite')
c = conn.cursor

In [19]:
df = pd.read_sql("select country,description,rating,price,province,title,winery from wine_data where variety = 'Chardonnay'", conn)
df.head(2)

,country,description,rating,price,province,title,winery
0,Portugal,A wood-aged Chardonnay from mountain vineyards...,91,25.0,Duriense,Real Companhia Velha 2009 Quinta de Cidrô Rese...,Real Companhia Velha
1,Portugal,This oak-fermented wine may have little connec...,88,25.0,Duriense,Real Companhia Velha 2014 Quinta de Cidrô Char...,Real Companhia Velha


In [10]:
year_column = 'title'
text = 'description'
labels = ['country','province','title','winery']
target = 'rating'
combine_text = ['country','province','title','winery', 'description']
numeric= ['price', 'year', 'word_count','country','province','title','winery']


y = df['rating']

In [11]:
def extract_year(dataframe):
    global year_column
    l = []
    i = 0 
    for year in range(len(dataframe)):
        temp = re.findall(r'\d+', dataframe[year_column][i]) 
        res = list(map(int, temp)) 
        try: 
            if len(str(res[0])) == 4:
                l.append(res[0])
            elif len(str(res[0])) != 4:
                l.append(0)
        except:
            l.append(0)
        #print(res[0])
        i+=1
    dataframe['year'] = l
    
    return dataframe

#add_year(df, df['title'])
#get_year = FunctionTransformer(extract_year, validate=False)
#get_year.fit_transform(df)
#add_year(df)

In [12]:
def word_count(dataframe):
    global text
    dataframe['word_count'] = dataframe[text].apply(lambda word: len(str(word).split(" ")))
    return dataframe

# word_count(df, df['description'])
# df.head()

# get_word_count = FunctionTransformer(word_count, validate=False)
# get_word_count.fit_transform(df)

In [13]:
# encoder = ce.JamesSteinEncoder(cols=[...]) --maybe (best score)
# encoder = ce.LeaveOneOutEncoder(cols=[...]) --maybe
# encoder = ce.MEstimateEncoder(cols=[...]) --maybe (good)
# encoder = ce.OrdinalEncoder(cols=[...]) --maybe
# encoder = ce.TargetEncoder(cols=[...]) --maybe

def category_encode(dataframe):
    global labels
    global target
    x = dataframe[labels]
    y = dataframe[target]
    ce_ord = ce.JamesSteinEncoder(cols=labels)
    dataframe = ce_ord.fit_transform(x, y)
    return dataframe
# get_encoded_text = FunctionTransformer(category_encode, validate=False)
# get_encoded_text.fit_transform(df)

In [ ]:
def combine_text_columns(dataframe):
    global combine_text
    text_data = dataframe[combine_text]
    
    # replace nans with blanks
    text_data.fillna("", inplace=True)
    
    # joins all of the text items in a row (axis=1)
    # with a space in between
    dataframe['text'] = text_data.apply(lambda x: " ".join(x).lower(), axis=1)
    
    return dataframe
#combine_text_columns(df)

In [14]:
# df['word_count'] = df['description'].apply(lambda word: len(str(word).split(" ")))
# df.head()

In [ ]:
# from sklearn import linear_model
# reg = linear_model.LinearRegression()
# reg.fit(X_train, y_train)
                                      
# # LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
# #                  normalize=False)
# reg.coef_
# reg.score(X_test, y_test)

In [23]:
from sklearn.preprocessing import FunctionTransformer

get_year = FunctionTransformer(extract_year, validate=False)
get_year.fit_transform(df)

get_word_count = FunctionTransformer(word_count, validate=False)
get_word_count.fit_transform(df)

get_encoded_text = FunctionTransformer(category_encode, validate=False)
get_encoded_text.fit_transform(df)


get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[numeric], validate=False)
get_numeric_data.fit_transform(df)

,price,year,word_count
0,25.0,2009,32
1,25.0,2014,49
2,20.0,2008,25
3,10.0,2013,58
4,19.0,2013,54
...,...,...,...
10229,15.0,2006,35
10230,37.0,2013,61
10231,8.0,2006,36
10232,17.0,2010,28


In [27]:
from sklearn.model_selection import train_test_split
features = df.drop('rating', axis=1)
X = features
y = df['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y
                                                   , test_size = .3
                                                   #, stratify=y
                                                  )
#features

In [32]:
pl = Pipeline(memory=None,
         steps=[
                    ('year', get_year),
                    ('word_count', get_word_count),
                    ('encode', get_encoded_text),
                    ('selector', get_numeric_data),
                    #('imputer', Imputer()),
                    #('int', SparseInteractions(degree=2)),
                    #('scale', MaxAbsScaler()),
                    ('clf', OneVsRestClassifier(LogisticRegression()))
               ], verbose=False)
pl.fit(X_train, y_train)

KeyError: 0

In [30]:
# set a reasonable number of features before adding interactions
chi_k = 300

# create the pipeline object
pl = Pipeline([
            transformer_list = [
                ('numeric_features', Pipeline([('year', get_year),
                    ('word_count', get_word_count),
                    ('encode', get_encoded_text),
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())]
                                             ))],
                ('int', SparseInteractions(degree=2)),
                ('scale', MaxAbsScaler()),
                ('clf', OneVsRestClassifier(LogisticRegression()))                          
                                              ])

pl.fit(X_train, y_train)

SyntaxError: invalid syntax (<ipython-input-30-5ad95b5d2142>, line 6)

In [ ]:
# set a reasonable number of features before adding interactions
chi_k = 300

# create the pipeline object
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('year', get_year),
                    ('word_count', get_word_count),
                    ('encode', get_encoded_text),
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', HashingVectorizer(token_pattern=TOKENS_ALPHANUMERIC,
                                                     norm=None, binary=False,
                                                     ngram_range=(1, 2))),
                    ('dim_red', SelectKBest(chi2, chi_k))
                ]))
             ]
        )),
        ('int', SparseInteractions(degree=2)),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

# fit the pipeline to our training data
pl.fit(X_train, y_train)


In [29]:
from itertools import combinations

import numpy as np
from scipy import sparse
from sklearn.base import BaseEstimator, TransformerMixin


class SparseInteractions(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2, feature_name_separator="_"):
        self.degree = degree
        self.feature_name_separator = feature_name_separator

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not sparse.isspmatrix_csc(X):
            X = sparse.csc_matrix(X)

        if hasattr(X, "columns"):
            self.orig_col_names = X.columns
        else:
            self.orig_col_names = np.array([str(i) for i in range(X.shape[1])])

        spi = self._create_sparse_interactions(X)
        return spi

    def get_feature_names(self):
        return self.feature_names

    def _create_sparse_interactions(self, X):
        out_mat = []
        self.feature_names = self.orig_col_names.tolist()

        for sub_degree in range(2, self.degree + 1):
            for col_ixs in combinations(range(X.shape[1]), sub_degree):
                # add name for new column
                name = self.feature_name_separator.join(self.orig_col_names[list(col_ixs)])
                self.feature_names.append(name)

                # get column multiplications value
                out = X[:, col_ixs[0]]
                for j in col_ixs[1:]:
                    out = out.multiply(X[:, j])

                out_mat.append(out)

        return sparse.hstack([X] + out_mat)